In [1]:
from pyomo.environ import *

In [2]:
demand = [30.0, 15.0, 15.0, 25.0, 33.0, 40.0,
          45.0, 45.0, 26.0, 14.0, 25.0, 30.0]

cost_normal = 32 *1000
cost_overtime = 40 * 1000
cost_store = 5 * 1000
capacity = 30
n = len(demand)

In [3]:
model = ConcreteModel(name="Bicycle")
model.time = RangeSet(1,n)
model.store_time = RangeSet(0,n)
model.prod_normal = Var(model.time, bounds=(0,capacity))
model.prod_overtime = Var(model.time, bounds=(0,capacity / 2))
model.store = Var(model.store_time, within=NonNegativeReals)

model.obj = Objective(expr=sum(cost_normal*model.prod_normal[i] +
                               cost_overtime*model.prod_overtime[i] +
                               cost_store*model.store[i] for i in model.time),
                      sense=minimize)

def time_constraint(model, t):
    inflow = model.prod_normal[t] + model.prod_overtime[t] + model.store[t-1]
    outflow = demand[t-1] + model.store[t]
    return inflow == outflow

model.constr = Constraint(model.time, rule=time_constraint)
model.store[0].fix(2.0)

In [4]:
solver = SolverFactory("glpk")
results = solver.solve(model)
print(results)


Problem: 
- Name: unknown
  Lower bound: 11247000.0
  Upper bound: 11247000.0
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 37
  Number of nonzeros: 48
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04414820671081543
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [5]:
model.pprint()

2 RangeSet Declarations
    store_time : Dim=0, Dimen=1, Size=13, Domain=Integers, Ordered=True, Bounds=(0, 12)
        Virtual
    time : Dim=0, Dimen=1, Size=12, Domain=Integers, Ordered=True, Bounds=(1, 12)
        Virtual

3 Var Declarations
    prod_normal : Size=12, Index=time
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  28.0 :    30 : False : False :  Reals
          2 :     0 :  15.0 :    30 : False : False :  Reals
          3 :     0 :  15.0 :    30 : False : False :  Reals
          4 :     0 :  28.0 :    30 : False : False :  Reals
          5 :     0 :  30.0 :    30 : False : False :  Reals
          6 :     0 :  30.0 :    30 : False : False :  Reals
          7 :     0 :  30.0 :    30 : False : False :  Reals
          8 :     0 :  30.0 :    30 : False : False :  Reals
          9 :     0 :  26.0 :    30 : False : False :  Reals
         10 :     0 :  14.0 :    30 : False : False :  Reals
         11 :     0 :  25.0 :    30 : False :

In [6]:
print("demand\tnormal\tover\tstore")
for t in range(1, 13):
    print("{:4}\t{:4}\t{:4}\t{:4}".format(demand[t-1],
          value(model.prod_normal[t]),
          value(model.prod_overtime[t]), value(model.store[t])))

demand	normal	over	store
30.0	28.0	 0.0	 0.0
15.0	15.0	 0.0	 0.0
15.0	15.0	 0.0	 0.0
25.0	28.0	 0.0	 3.0
33.0	30.0	 0.0	 0.0
40.0	30.0	10.0	 0.0
45.0	30.0	15.0	 0.0
45.0	30.0	15.0	 0.0
26.0	26.0	 0.0	 0.0
14.0	14.0	 0.0	 0.0
25.0	25.0	 0.0	 0.0
30.0	30.0	 0.0	 0.0
